In [21]:
import os
import sys
from os.path import join
import json

# Supress future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import nfl_data_py as nfl

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, os.path.join(ROOT_DIR,'py'))

import util
from plot.plot_simple import plot_play_with_speed

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

with open("paths.json", 'r') as f:
    paths = json.load(f)

PROCESSED_DATA_PATH = paths['processed_data']

In [22]:
run_concepts = pd.read_pickle(join(PROCESSED_DATA_PATH, 'run_concepts.pkl'))

motion = pd.DataFrame()
for wk in range(1,10):
    tmp = pd.read_pickle(join(PROCESSED_DATA_PATH, f'wk{wk}', 'motion_plays.pkl'))
    motion = pd.concat([motion, tmp])
motion = motion.reset_index(drop=True)
motion['motion_group'] = motion.motion_group.fillna('DROP')

df = run_concepts.merge(motion, on='game_play_id', how='left')

df = df[df.motion_group != 'DROP']

del tmp, motion, run_concepts

In [23]:
df['motion_towards_playside'] = np.where(
    df.play_dir_location.isin(['inside-left', 'outside-left']) & (df.motion_dir_last == 'left'),
    1,
    np.where(
        df.play_dir_location.isin(['inside-right', 'outside-right']) & (df.motion_dir_last == 'right'),
        -1,
        np.where(
            df.play_dir_location == 'middle',
            0,
            np.nan
        )
    )
)

drop_cols = ['play_dir', 'play_dir_location', 'motion_nfl_id', 'motion_position',
             'initial_alignment', 'n_direction_changes', 'same_motion_dir',
             'motion_dir_first', 'motion_dir_last', 'motion_sub_group']
for c in drop_cols:
    if c in df.columns:
        df.drop(columns=c, inplace=True)

In [24]:
cols = ['game_play_id',
       # 'down', 'yards_to_go', 
       # 'possession_team', 'defensive_team', 
       'absolute_yardline_number', 
       #'pre_snap_home_team_win_probability', 'pre_snap_visitor_team_win_probability', 
       'offense_formation', 'defenders_in_box', 
       'expected_points_added']

df_play = pd.DataFrame()
for wk in range(1, 10):
    tmp = pd.read_pickle(join(PROCESSED_DATA_PATH, f'wk{wk}', 'play_final.pkl'))
    df_play = pd.concat([df_play, tmp[cols]])

df_play = df_play.reset_index(drop=True)

del tmp

In [25]:
df = df.merge(df_play, on='game_play_id', how='left')

In [26]:
# Drop "DRAW" plays as they dont fall into Zone, Gap, or Man concepts
df = df.query('run_concept != "DRAW"').reset_index(drop=True)

df['motion_present'] = np.where(df.motion_group.isna(), 0, 1)
df.query('motion_present == 1').value_counts('run_concept')

run_concept
OUTSIDE ZONE    624
INSIDE ZONE     380
MAN             310
POWER           202
PULL LEAD       187
COUNTER         119
TRAP             22
dtype: int64

In [27]:
# Group Trap, Counter, Pull Lead, and Power plays into Gap
df['run_concept'] = np.where(
    df.run_concept.isin(['TRAP', 'COUNTER', 'PULL LEAD', 'POWER']),
    'GAP',
    df.run_concept
)
df.query('motion_present == 1').value_counts('run_concept')

run_concept
OUTSIDE ZONE    624
GAP             530
INSIDE ZONE     380
MAN             310
dtype: int64

In [28]:
(
    df
    .value_counts(['run_concept', 'motion_group'])
    .reset_index().rename(columns={0:'count'})
    .sort_values(['run_concept', 'count'], ascending=[True, False])
)

,run_concept,motion_group,count
1,GAP,Jet,165
5,GAP,Over,104
7,GAP,Orbit,88
9,GAP,Fly,65
10,GAP,Glide,63
17,GAP,Yo-Yo,41
27,GAP,Shuffle,2
28,GAP,In,2
2,INSIDE ZONE,Jet,155
8,INSIDE ZONE,Orbit,77


In [29]:
# Drop Shuffle and In motion plays, as there are too few instances to ensure the distribution is representative
df = df.query('motion_group not in ["Shuffle", "In"]').reset_index(drop=True)

In [30]:
(
    df
    .value_counts(['run_concept', 'motion_group'])
    .reset_index().rename(columns={0:'count'})
    .sort_values(['run_concept', 'count'], ascending=[True, False])
)

,run_concept,motion_group,count
1,GAP,Jet,165
5,GAP,Over,104
7,GAP,Orbit,88
9,GAP,Fly,65
10,GAP,Glide,63
17,GAP,Yo-Yo,41
2,INSIDE ZONE,Jet,155
8,INSIDE ZONE,Orbit,77
13,INSIDE ZONE,Fly,50
19,INSIDE ZONE,Yo-Yo,36
